In [1]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

raw_data = load_wine()

df_features = pd.DataFrame(raw_data.data, columns=raw_data.feature_names)
df_target = pd.Series(raw_data.target, name='target')

print("Kích thước dữ liệu:", df_features.shape)
print("\n5 dòng đầu tiên của dữ liệu (Features):")
print(df_features.head())

print("\nCác loại nhãn (Target) cần dự đoán:", df_target.unique())


x_train, x_test, y_train, y_test = train_test_split(df_features, df_target, test_size=0.2, random_state=42)
print(f"Train ({len(x_train)} dòng), Test ({len(x_test)} dòng)")

Kích thước dữ liệu: (178, 13)

5 dòng đầu tiên của dữ liệu (Features):
   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  

In [2]:
# Sử dụng mô hình Decision Tree đơn lẻ

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


def decisionTree(x_train, y_train, x_test, y_test) -> float:

    tree_model = DecisionTreeClassifier(random_state=42)
    tree_model.fit(x_train, y_train)

    # Đưa dữ liệu đề thi (X_test) để mô hình tự đoán
    y_pred_tree = tree_model.predict(x_test)

    return y_pred_tree



In [8]:
import numpy as np
from scipy import stats
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

k = 10000
all_predictions = []

print(f"Bắt đầu quy trình Manual Bagging với {k} cây...")

for i in range(k):
    x_boot, y_boot = resample(x_train, y_train, replace=True, random_state=i)
    
    print(f"Huấn luyện cây thứ {i+1}...", end =' ')
    y_pred_tree = decisionTree(x_boot, y_boot, x_test, y_test)
    
    # Lưu list lá phiếu
    all_predictions.append(y_pred_tree)
    
    acc_this_tree = accuracy_score(y_test, y_pred_tree)
    print(f"Epoch {i+1}: Tree Accuracy = {acc_this_tree*100:.2f}%")




all_predictions = np.array(all_predictions)

# Bầu cử đa số (Majority Voting)
# mode sẽ tính dọc theo trục 0 (dọc theo các cây) xem nhãn nào xuất hiện nhiều nhất
final_pred_result = stats.mode(all_predictions, axis=0, keepdims=True)

# Lấy mảng kết quả cuối cùng (hàm mode trả về 1 object gồm mode và count, ta chỉ lấy mode)
final_pred = final_pred_result.mode[0] 

acc_manual = accuracy_score(y_test, final_pred)
print("-" * 30)
print(f"Độ chính xác tổng hợp của Manual Bagging: {acc_manual*100:.2f}%")

Bắt đầu quy trình Manual Bagging với 10000 cây...
Huấn luyện cây thứ 1... Epoch 1: Tree Accuracy = 97.22%
Huấn luyện cây thứ 2... Epoch 2: Tree Accuracy = 94.44%
Huấn luyện cây thứ 3... Epoch 3: Tree Accuracy = 97.22%
Huấn luyện cây thứ 4... Epoch 4: Tree Accuracy = 88.89%
Huấn luyện cây thứ 5... Epoch 5: Tree Accuracy = 94.44%
Huấn luyện cây thứ 6... Epoch 6: Tree Accuracy = 97.22%
Huấn luyện cây thứ 7... Epoch 7: Tree Accuracy = 91.67%
Huấn luyện cây thứ 8... Epoch 8: Tree Accuracy = 91.67%
Huấn luyện cây thứ 9... Epoch 9: Tree Accuracy = 100.00%
Huấn luyện cây thứ 10... Epoch 10: Tree Accuracy = 91.67%
Huấn luyện cây thứ 11... Epoch 11: Tree Accuracy = 88.89%
Huấn luyện cây thứ 12... Epoch 12: Tree Accuracy = 91.67%
Huấn luyện cây thứ 13... Epoch 13: Tree Accuracy = 94.44%
Huấn luyện cây thứ 14... Epoch 14: Tree Accuracy = 86.11%
Huấn luyện cây thứ 15... Epoch 15: Tree Accuracy = 94.44%
Huấn luyện cây thứ 16... Epoch 16: Tree Accuracy = 94.44%
Huấn luyện cây thứ 17... Epoch 17: Tree

In [9]:
# Code thư viện

from sklearn.ensemble import BaggingClassifier

# 1. Khởi tạo mô hình Bagging
# - estimator: Bạn đưa cái cây đơn lẻ vào đây để làm mẫu (base learner)
# - n_estimators: Số lượng cây bạn muốn tạo (đề bài yêu cầu 100)
# - random_state: Cố định sự ngẫu nhiên để kết quả không nhảy lung tung
print("Đang huấn luyện Bagging (100 cây)...")

bagging_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42), 
    n_estimators=100, 
    random_state=42
)

# 2. Học (Fit)
# Hàm này tự động làm các việc: 
# - Bootstrap lấy mẫu 100 lần
# - Xây 100 cây riêng biệt
bagging_model.fit(x_train, y_train)

# 3. Thi (Predict)
# 100 cây sẽ cùng bỏ phiếu cho từng mẫu trong tập Test
y_pred_bagging = bagging_model.predict(x_test)
# 4. Chấm điểm
acc_bagging = accuracy_score(y_test, y_pred_bagging)
print(f"Độ chính xác của Bagging (100 cây): {acc_bagging:.2%}")

Đang huấn luyện Bagging (100 cây)...
Độ chính xác của Bagging (100 cây): 97.22%


In [10]:
# Nhận xét


# - So sánh độ chính xác của Manual Bagging và Bagging từ thư viện sklearn
# - Nhận xét về sự khác biệt (nếu có) giữa hai kết quả
print("\nNhận xét:")
print(f"- Độ chính xác Code tay Bagging: {acc_manual*100:.2f}%")
print(f"- Độ chính xác Bagging từ sklearn: {acc_bagging*100:.2f}%")

if abs(acc_manual - acc_bagging) < 1e-6:
    print("- Kết quả gần như giống hệt nhau, chứng tỏ quá trình Bagging thủ công đã được thực hiện chính xác.")


Nhận xét:
- Độ chính xác Code tay Bagging: 97.22%
- Độ chính xác Bagging từ sklearn: 97.22%
- Kết quả gần như giống hệt nhau, chứng tỏ quá trình Bagging thủ công đã được thực hiện chính xác.
